In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from utils import forward, obj, penalty, grad

Computational Domain

In [ ]:
alpha = 0.5
beta = 0.00001
n_grid = 100
n_t = int(n_grid / 1.8)  # CFL = 0.9
step_grid = 2 / (n_grid - 1)
step_t = 1 / (n_t - 1)  # T = 1
x = np.linspace(-1, 1, n_grid)

Target Parameters in shape (n_grid,)

In [ ]:
g_data = 0.5 * np.ones(n_grid)
for i in range(n_grid):
    if -0.5 <= i * step_grid - 1 <= 0:
        g_data[i] = 1
u_data = forward(g_data, n_t, step_t, n_grid, step_grid)

Adjoint & Sequential Least Squares Programming

In [ ]:
g0 = 0.5 * np.ones(n_grid)
for i in range(100):
    gradient = grad(g0, n_t, step_t, n_grid, step_grid, u_data, alpha, beta)
    norm_grad = np.linalg.norm(gradient)
    if norm_grad < 1e-3:
        break
    else:
        g0 = g0 - 0.1 * gradient / norm_grad
        print(f'Iteration {i}, ||grad|| = {np.max(gradient / norm_grad)}')

In [ ]:
g_res = g0
u_res = forward(g_res, n_t, step_t, n_grid, step_grid)
print(f'J: {obj(u_res, u_data, step_grid) + penalty(g_res, n_grid, step_grid, beta)}')
print(norm_grad)

plt.subplot(1, 2, 1)
plt.plot(x, g_data, label='$g_d$')
plt.plot(x, g_res, label='$g_{adjoint}$')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(x, u_data, label='$u_d$')
plt.plot(x, u_res, label='$u_{adjoint}$')
plt.legend()

plt.suptitle(f'Grid:{n_grid}')
plt.show()